In [1]:
import numpy as np
from astropy import constants as const
from astropy.coordinates import Angle
from astropy import units
import textwrap
wrap_len = 100

In [2]:
max_decorr = 0.1
frequency = (250. * 1e6 * units.Hz)
wavelength = const.c / frequency.to(1/units.s)
max_baseline_m = (870 * units.m)
earth_rot_speed = (Angle(360, units.deg) / units.sday).to(units.arcminute/units.s)
corr_FoV_min = Angle(10., units.degree)
corr_FoV_max = Angle(90., units.degree)
hera_latitude = Angle('-30:43:17.5', units.deg)

In [3]:
max_resolution = Angle(np.arcsin(wavelength/max_baseline_m), units.radian)
print(textwrap.fill('Max resolution', width=wrap_len))
max_resolution.to(units.arcminute)

Max resolution


<Angle 4.738440830953493 arcmin>

In [4]:
integration_time = (max_resolution * max_decorr).to(units.arcminute)/earth_rot_speed
print(textwrap.fill('Max integration time required to keep the decorrelation due to time integrating '
      'under max_decorr on the longest baselines', width=wrap_len))
integration_time

Max integration time required to keep the decorrelation due to time integrating under max_decorr on
the longest baselines


<Quantity 1.8902011330061352 s>

In [5]:
channel_width_max = ((const.c * max_decorr) / (max_baseline_m*np.sin(corr_FoV_min.to(units.rad)))).to(units.kHz)
print(textwrap.fill('Max channel width to keep the decorrelation due to channel width under max_decorr '
      'for a 10 degree correlator FoV on the longest baselines', width=wrap_len))
channel_width_max

Max channel width to keep the decorrelation due to channel width under max_decorr for a 10 degree
correlator FoV on the longest baselines


<Quantity 198.44091473557216 kHz>

In [6]:
channel_width_min = ((const.c * max_decorr) / (max_baseline_m*np.sin(corr_FoV_max.to(units.rad)))).to(units.kHz)
print(textwrap.fill('Max channel width to keep the decorrelation due to channel width under max_decorr '
      'for a 90 degree correlator FoV on the longest baselines', width=wrap_len))
channel_width_min

Max channel width to keep the decorrelation due to channel width under max_decorr for a 90 degree
correlator FoV on the longest baselines


<Quantity 34.45890321839081 kHz>

In [7]:
# After fringe stopping, the rotation is from the sky rotating in the beam
# This is slower, so we can sum in time to longer integrations
# (this summing does cause more decorrelation but it's better than without fringe stopping and it decreases data rates)
fringe_stopped_int_time_max = ((max_resolution.to(units.arcminute) * max_decorr)/
                               (np.sin(corr_FoV_min.radian) * earth_rot_speed * abs(np.sin(hera_latitude))))
print(textwrap.fill('Max integration time to keep the decorrelation due to time '
                    'integrating after fringe stopping under max_decorr for a 10 '
                    'degree correlator FoV on the longest baselines', width=wrap_len))
fringe_stopped_int_time_max

Max integration time to keep the decorrelation due to time integrating after fringe stopping under
max_decorr for a 10 degree correlator FoV on the longest baselines


<Quantity 21.307418105017184 s>

In [8]:
fringe_stopped_int_time_min = ((max_resolution.to(units.arcminute) * max_decorr) /
                               (np.sin(corr_FoV_max.radian) * earth_rot_speed * abs(np.sin(hera_latitude))))
print(textwrap.fill('Max integration time to keep the decorrelation due to time '
                    'integrating after fringe stopping under max_decorr '
                    'for a 90 degree correlator FoV on the longest baselines', width=wrap_len))
fringe_stopped_int_time_min

Max integration time to keep the decorrelation due to time integrating after fringe stopping under
max_decorr for a 90 degree correlator FoV on the longest baselines


<Quantity 3.6999943247235914 s>

Now invert these equations to calculate the decorrelation caused by some suggested correlator settings

In [9]:
# This is the max number of channels that Jack can do 'easily'
n_channels = 4096
corr_chan_width = (250 * units.MHz) / n_channels
# Since we will integrate longer after fringe stopping, there's no need to integrate more here
corr_integration_time = .1 * units.s
# after fringe stopping, we can sum in time to longer integrations
corr_post_fringe_stop_int_time = 10 * units.s
print('proposed channel width: ', corr_chan_width.to(units.kHz))
print('proposed pre fringe stopping integration time: ', corr_integration_time.to(units.ms))
print('proposed post fringe stopping integration time: ', corr_post_fringe_stop_int_time)

proposed channel width:  61.03515625 kHz
proposed pre fringe stopping integration time:  100.0 ms
proposed post fringe stopping integration time:  10.0 s


In [10]:
decorr_int_time = corr_integration_time * earth_rot_speed / max_resolution.to(units.arcminute)
print(textwrap.fill('Decorrelation fraction due to integration time on the longest baseline', width=wrap_len))
decorr_int_time

Decorrelation fraction due to integration time on the longest baseline


<Quantity 0.005290442284359557>

In [11]:
decorr_chan_width_max = corr_chan_width.to(1/units.s) * max_baseline_m * np.sin(corr_FoV_max.to(units.rad)) / const.c 
print(textwrap.fill('Decorrelation due to channel width for a 90 degree '
                    'correlator FoV for the longest baseline', width=wrap_len))
decorr_chan_width_max

Decorrelation due to channel width for a 90 degree correlator FoV for the longest baseline


<Quantity 0.17712448902733904>

In [12]:
decorr_chan_width_min = corr_chan_width.to(1/units.s) * max_baseline_m * np.sin(corr_FoV_min.to(units.rad)) / const.c 
print(textwrap.fill('Decorrelation due to channel width for a 10 degree '
                    'correlator FoV for the longest baseline', width=wrap_len))
decorr_chan_width_min

Decorrelation due to channel width for a 10 degree correlator FoV for the longest baseline


<Quantity 0.030757344739783626>

In [13]:
corr_FoV_max_decorr = Angle(np.arcsin(min([1, (const.c * max_decorr) / 
                                              (max_baseline_m * corr_chan_width.to(1/units.s))])), units.radian)
print(textwrap.fill('Correlator FoV with decorrelation due to channel '
                    'width under max_decorr (10%) for the longest baseline', width=wrap_len))
corr_FoV_max_decorr.to(units.degree)

Correlator FoV with decorrelation due to channel width under max_decorr (10%) for the longest
baseline


<Angle 34.372760870741885 deg>

In [14]:
decorr_fringe_stop_max = (corr_post_fringe_stop_int_time * np.sin(corr_FoV_max.radian) * earth_rot_speed * 
                          abs(np.sin(hera_latitude))) / max_resolution.to(units.arcminute)
print(textwrap.fill('Decorrelation due to time integrating after fringe '
                    'stopping for a 90 degree correlator FoV for the longest baseline', width=wrap_len))
decorr_fringe_stop_max

Decorrelation due to time integrating after fringe stopping for a 90 degree correlator FoV for the
longest baseline


<Quantity 0.27027068482725447>

In [15]:
decorr_fringe_stop_min = (corr_post_fringe_stop_int_time * np.sin(corr_FoV_min.radian) * earth_rot_speed * 
                          abs(np.sin(hera_latitude))) / max_resolution.to(units.arcminute)
print(textwrap.fill('correlation due to time integrating after fringe '
                    'stopping for a 10 degree correlator FoV for the longest baseline', width=wrap_len))
decorr_fringe_stop_min

correlation due to time integrating after fringe stopping for a 10 degree correlator FoV for the
longest baseline


<Quantity 0.046932011897046017>

In [16]:
corr_FoV_fringe_stop_max_decorr =  Angle(np.arcsin(min([1, max_resolution.to(units.arcminute) * max_decorr / 
                                                   (corr_post_fringe_stop_int_time * earth_rot_speed * 
                                                    abs(np.sin(hera_latitude)))])), units.radian)
print(textwrap.fill('Correlator FoV with decorrelation just due to time '
                    'integrating after fringe stopping under max_decorr (10%) for the longest baseline', width=wrap_len))
corr_FoV_fringe_stop_max_decorr.to(units.degree)

Correlator FoV with decorrelation just due to time integrating after fringe stopping under
max_decorr (10%) for the longest baseline


<Angle 21.715582282378335 deg>

Decorrelation factors actually need to be combined

In [17]:
pre_fs_total_decorr_max = 1- (1-decorr_int_time) * (1-decorr_chan_width_max)
print(textwrap.fill('Net decorrelation before fringe stopping due to integration '
                    'time and channel width for 90 degree correlator FoV for the longest baseline', width=wrap_len))
pre_fs_total_decorr_max

Net decorrelation before fringe stopping due to integration time and channel width for 90 degree
correlator FoV for the longest baseline


<Quantity 0.18147786442535274>

In [18]:
pre_fs_total_decorr_min = 1- (1-decorr_int_time) * (1-decorr_chan_width_min)
print(textwrap.fill('Net decorrelation before fringe stopping due to integration '
                    'time and channel width for 10 degree correlator FoV for the longest baseline', width=wrap_len))
pre_fs_total_decorr_min

Net decorrelation before fringe stopping due to integration time and channel width for 10 degree
correlator FoV for the longest baseline


<Quantity 0.03588506706697725>

In [19]:
pre_fs_corr_FoV_max_decorr = Angle(np.arcsin(min([1, ((1-(1-max_decorr)/(1-decorr_int_time)) * const.c) / 
                                              (max_baseline_m * corr_chan_width.to(1/units.s))])), units.radian)
print(textwrap.fill('Correlator FoV before fringe stopping with decorrelation due to '
                    'integration time and channel width under max_decorr (10%) for the longest baseline', width=wrap_len))
pre_fs_corr_FoV_max_decorr.to(units.degree)

Correlator FoV before fringe stopping with decorrelation due to integration time and channel width
under max_decorr (10%) for the longest baseline


<Angle 32.51701540992716 deg>

In [20]:
post_fs_total_decorr_max = 1-(1-pre_fs_total_decorr_max) * (1-decorr_fringe_stop_max)
print(textwrap.fill('Total decorrelation after fringe stopping and '
                    'integrating for a 90 degree correlator FoV', width=wrap_len))
post_fs_total_decorr_max

Total decorrelation after fringe stopping and integrating for a 90 degree correlator FoV


<Quantity 0.40270040255337947>

In [21]:
post_fs_total_decorr_min = 1-(1-pre_fs_total_decorr_min) * (1-decorr_fringe_stop_min)
print(textwrap.fill('Total decorrelation after fringe stopping and '
                    'integrating for a 10 degree correlator FoV', width=wrap_len))
post_fs_total_decorr_min

Total decorrelation after fringe stopping and integrating for a 10 degree correlator FoV


<Quantity 0.0811329205695096>

In [22]:
# calculate correlator FoV after fringe stopping and integrating with
# total decorrelation under max_decorr (10%) for the longest baseline
# This is a quadratic in sin(FoV), so takes some setup
# form: a_term * sin(FoV)**2 + b_term * sin(FoV) + c_term
a_term = (max_baseline_m * corr_chan_width.to(1/units.s) * corr_post_fringe_stop_int_time * 
          earth_rot_speed * abs(np.sin(hera_latitude)))/(const.c * max_resolution.to(units.arcminute))
b_term = (-1) * (max_baseline_m * corr_chan_width.to(1/units.s)/const.c + 
                 corr_post_fringe_stop_int_time * earth_rot_speed * abs(np.sin(hera_latitude))/
                 (max_resolution.to(units.arcminute)))
c_term = 1 - (1-max_decorr)/(1-decorr_int_time)
sin_fov = ((-1) * b_term - np.sqrt(b_term**2 - 4 * a_term * c_term))/(2 * a_term)
post_fs_corr_FoV_max_decorr = Angle(np.arcsin(sin_fov), units.rad)
print(textwrap.fill('Correlator FoV after fringe stopping and integrating with '
                    'total decorrelation under max_decorr (10%) for the longest baseline', width=wrap_len))
post_fs_corr_FoV_max_decorr.to(units.degree)

Correlator FoV after fringe stopping and integrating with total decorrelation under max_decorr (10%)
for the longest baseline


<Angle 12.585558849043819 deg>

Data rate calculations:

In [23]:
n_antennas = 350
channels_to_keep = n_channels * 3/4
sum_diff_factor = 2
bytes_per_vis = (8 * units.byte)
n_polarizations = 4

In [24]:
Naive_data_rate = (channels_to_keep * (n_antennas*(n_antennas+1)/2) * n_polarizations * 
                   bytes_per_vis * sum_diff_factor  / corr_integration_time)
print('Naive (correlator internal):')
print('data rate: ', Naive_data_rate.to(units.Gbyte/units.s))
print('channel width: ', corr_chan_width.to(units.kHz))
print('integration time: ', corr_integration_time)

Naive (correlator internal):
data rate:  120.76646400000001 Gbyte / s
channel width:  61.03515625 kHz
integration time:  0.1 s


In [25]:
post_fringe_stop_data_rate = (channels_to_keep * (n_antennas*(n_antennas+1)/2) * n_polarizations * 
                              bytes_per_vis * sum_diff_factor  / corr_post_fringe_stop_int_time)
print('post fringe stopping:')
print('data rate: ', post_fringe_stop_data_rate.to(units.Gbyte/units.s))
print('channel width: ', corr_chan_width.to(units.kHz))
print('integration time: ', corr_post_fringe_stop_int_time)

post fringe stopping:
data rate:  1.2076646400000002 Gbyte / s
channel width:  61.03515625 kHz
integration time:  10.0 s
